In [16]:
import pandas as pd
import numpy as np
import os
import shapely
from shapely.geometry import Point, Polygon
import geopandas as gpd

print("done")


done


In [17]:
# Function to extract the numbers before the first underscore
def extract_numbers(value):
    return value.split('_')[0]

# List of files to process with full paths
files = [
    r'D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\may_meteo24.csv',
    r'D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\abr_meteo24.csv'
]

for file in files:
    # Read the CSV file
    df = pd.read_csv(file, sep=";")
    
    # Check if the column 'PUNTO_MUESTREO' exists in the DataFrame
    if 'PUNTO_MUESTREO' in df.columns:
        # Create the new column and fill it with the extracted values
        df['ID_EST'] = df['PUNTO_MUESTREO'].apply(extract_numbers)
        
        # Reorder columns to make the new column the first one
        columns_order = ['ID_EST'] + [col for col in df.columns if col != 'ID_EST']
        df = df[columns_order]
        #drop unnecessary cols
        df.drop(df.columns[[1, 2, 3]], axis=1, inplace=True)
    
        # Find columns that start with 'V'
        columns_with_v = [col for col in df.columns if col.startswith('V')]

        # Keep rows where any of these columns have the value 'V'
        # Variables not assigned a V correspond to not valid measurements
        for col in columns_with_v:
            df = df[df[col] == 'V']

        
        # Drop all validation columns 
        df = df.loc[:, ~df.columns.str.startswith('V')]
        # Reshape the DataFrame to have hourly data in rows
        df_melted = df.melt(id_vars=['ID_EST', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'],
                            var_name='HOUR', value_name='VALUE')
        
        # Extract the hour from the 'HOUR' column
        df_melted['HOUR'] = df_melted['HOUR'].str.extract('(\d+)', expand=False).astype(int)
        
        # Sort the DataFrame by ID_EST, MAGNITUD, and timestamp
        df_melted = df_melted.sort_values(by=['ID_EST', 'MAGNITUD', 'ANO', 'MES', 'DIA', 'HOUR'])
        # Save the modified DataFrame to a new CSV file
        output_file = file.replace('.csv', '_modified.csv')
        df_melted.to_csv(output_file, index=False)
        


In [18]:
#concatenate the modified files 
may = pd.read_csv('D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\may_meteo24_modified.csv',sep=",")
apr = pd.read_csv('D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\abr_meteo24_modified.csv',sep=",")
meteo_data = pd.concat([apr, may], ignore_index=True)
meteo_data

,ID_EST,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,HOUR,VALUE
0,28079004,83,28079004_83_98,2024,4,1,1,6.2
1,28079004,83,28079004_83_98,2024,4,1,2,6.1
2,28079004,83,28079004_83_98,2024,4,1,3,5.7
3,28079004,83,28079004_83_98,2024,4,1,4,5.4
4,28079004,83,28079004_83_98,2024,4,1,5,5.4
...,...,...,...,...,...,...,...,...
109363,28079115,86,28079115_86_98,2024,5,31,20,16.0
109364,28079115,86,28079115_86_98,2024,5,31,21,18.0
109365,28079115,86,28079115_86_98,2024,5,31,22,20.0
109366,28079115,86,28079115_86_98,2024,5,31,23,23.0


In [19]:
# Now we merge the air_data with the coordinates of each station
# Load the CSV file
file_path = "D:\\Data projects\\Air pollution Madrid\\data\\locations\\meteo_locations_modified.csv"
locations_df = pd.read_csv(file_path, sep=",")


# Convert the ID_EST column to int to match the ID column in locations_df
meteo_data['ID_EST'] = meteo_data['ID_EST'].astype(int)

# Ensure the ID column in locations_df is also of type int (it usually should be)
locations_df['ID'] = locations_df['ID'].astype(int)

# Merge the DataFrames on the corrected ID columns
merged_df = pd.merge(meteo_data, locations_df, left_on='ID_EST', right_on='ID', how='left')

# Specify the directory and filename to save the combined DataFrame
output_directory = r"D:\\Data projects\\Air pollution Madrid\\data\\meteo data"
output_filename = 'meteo_geolocated.csv'
output_file = os.path.join(output_directory, output_filename)

# Save the combined DataFrame to a new CSV file
merged_df.to_csv(output_file, index=False)

print("This is the resulting dataset")
merged_df

This is the resulting dataset


,ID_EST,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,HOUR,VALUE,ID,LATITUD,LONGITUD
0,28079004,83,28079004_83_98,2024,4,1,1,6.2,28079004,404.238.823,-37.122.567
1,28079004,83,28079004_83_98,2024,4,1,2,6.1,28079004,404.238.823,-37.122.567
2,28079004,83,28079004_83_98,2024,4,1,3,5.7,28079004,404.238.823,-37.122.567
3,28079004,83,28079004_83_98,2024,4,1,4,5.4,28079004,404.238.823,-37.122.567
4,28079004,83,28079004_83_98,2024,4,1,5,5.4,28079004,404.238.823,-37.122.567
...,...,...,...,...,...,...,...,...,...,...,...
109363,28079115,86,28079115_86_98,2024,5,31,20,16.0,28079115,403.925.444,-3.697.631
109364,28079115,86,28079115_86_98,2024,5,31,21,18.0,28079115,403.925.444,-3.697.631
109365,28079115,86,28079115_86_98,2024,5,31,22,20.0,28079115,403.925.444,-3.697.631
109366,28079115,86,28079115_86_98,2024,5,31,23,23.0,28079115,403.925.444,-3.697.631


In [20]:
132888/6

22148.0

In [21]:
# Drop rows with NaN values in MAGNITUD column
merged_df.dropna(subset=['MAGNITUD'], inplace=True)

NAS = merged_df["MAGNITUD"].isnull().sum()
print(f"Number of NaN values in column 'MAGNITUD': {NAS}")


Number of NaN values in column 'MAGNITUD': 0


In [22]:
merged_df['MAGNITUD'] = merged_df['MAGNITUD'].astype(int)

# Mapping of MAGNITUD codes to parameter names
magnitude_mapping = {
    80: 'RADIACION_ULTRAVIOLETA',
    81: 'VELOCIDAD_VIENTO',
    82: 'DIR_DE_VIENTO',
    83: 'TEMPERATURA',
    86: 'HUMEDAD_RELATIVA',
    87: 'PRESION_BARIOMETRICA',
    88: 'RADIACION_SOLAR',
    89: 'PRECIPITACION'
}

# Replace MAGNITUD codes with their respective names
merged_df['MAGNITUD'] = merged_df['MAGNITUD'].map(magnitude_mapping)

# Pivot the table
meteo = merged_df.pivot_table(index=['ID_EST', 'ANO', 'MES', 'DIA', 'HOUR', 'LATITUD', 'LONGITUD'],
                                    columns='MAGNITUD',
                                    values='VALUE', aggfunc='mean').reset_index()

# Flatten the columns
meteo.columns.name = None

print("The meteo dataset is ready:")
meteo

The meteo dataset is ready:


,ID_EST,ANO,MES,DIA,HOUR,LATITUD,LONGITUD,DIR_DE_VIENTO,HUMEDAD_RELATIVA,PRECIPITACION,PRESION_BARIOMETRICA,RADIACION_SOLAR,TEMPERATURA,VELOCIDAD_VIENTO
0,28079004,2024,4,1,1,404.238.823,-37.122.567,NaN,NaN,NaN,NaN,NaN,6.2,NaN
1,28079004,2024,4,1,2,404.238.823,-37.122.567,NaN,NaN,NaN,NaN,NaN,6.1,NaN
2,28079004,2024,4,1,3,404.238.823,-37.122.567,NaN,NaN,NaN,NaN,NaN,5.7,NaN
3,28079004,2024,4,1,4,404.238.823,-37.122.567,NaN,NaN,NaN,NaN,NaN,5.4,NaN
4,28079004,2024,4,1,5,404.238.823,-37.122.567,NaN,NaN,NaN,NaN,NaN,5.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30499,28079115,2024,5,31,20,403.925.444,-3.697.631,NaN,16.0,NaN,NaN,NaN,30.8,NaN
30500,28079115,2024,5,31,21,403.925.444,-3.697.631,NaN,18.0,NaN,NaN,NaN,29.5,NaN
30501,28079115,2024,5,31,22,403.925.444,-3.697.631,NaN,20.0,NaN,NaN,NaN,26.8,NaN
30502,28079115,2024,5,31,23,403.925.444,-3.697.631,NaN,23.0,NaN,NaN,NaN,25.1,NaN
